In [229]:
import numpy as np
import math
pi = math.pi 


In [230]:

def H_table (tetha): # The Har
    return np.array([
        [(pi/2)+tetha[0], (pi/2), 0, 3],
        [(pi/2)+tetha[1], (pi/2), 0, 0],
        [tetha[2], (-1*pi/2), 3, 0],
        [tetha[3], 0, 2, 0]
        ])

In [231]:
def H_transformation (param):
    tetha = param[0]
    alpha = param[1]
    r = param[2]
    d = param[3]
    c_t = math.cos(tetha)
    s_t = math.sin(tetha)
    c_a = math.cos(alpha)
    s_a = math.sin(alpha)
    return np.array([
        [c_t, (-1*s_t*c_a), (s_t*s_a), (r*c_t)],
        [s_t, (c_t*c_a), (-1*c_t*s_a), (r*s_t)],
        [0, s_a, c_a, d], 
        [0,0,0,1]
        ])

In [232]:
def H_matrix(start, end):
    matrix = np.eye(4)
    for i in range(start, end):
        matrix = np.matmul (matrix, H_transformation(H_tab[i]))
    return matrix

In [233]:
def R(start, end):
    matrix = np.eye(4)
    for i in range (start+1, end+1):
        matrix = np.matmul(matrix, (H_transformation(H_tab[i-1])))
    return matrix [0:3,0:3]

In [234]:
def d(start, end):
    matrix = np.eye(4)
    for i in range (start+1, end+1):
        matrix = np.matmul(matrix, (H_transformation(H_tab[i-1])))
    return (matrix [0:3,3]).reshape((3,1))

In [239]:
def Jacobian():
    n = 4
    jacobian = []
    for i in range(0,4):
        my_vec = np.matmul(R(0,i), np.array([[0],[0],[1]])).transpose()[0]
        temp = np.cross(my_vec , (d(0,n)-d(0,i)).transpose()[0])
        jacobian.append(temp)
    jacobian = np.asarray(jacobian)
    jacobian = jacobian.transpose()
    return jacobian


In [251]:
tetha = [0,0,0,0]
H_tab = H_table(tetha)
# H1 = H_transformation(H_tab[0])
# H2 = H_transformation(H_tab[1])
# H3 = H_transformation(H_tab[2])
# H4 = H_transformation(H_tab[3])
print(Jacobian())

[[-3.0616170e-16 -3.0616170e-16  5.0000000e+00 -1.2246468e-16]
 [-3.0616170e-16 -5.0000000e+00 -6.1232340e-16 -2.0000000e+00]
 [ 0.0000000e+00  3.0616170e-16  3.0616170e-16  1.2246468e-16]]


In [252]:
delta = np.array([0,1,0,0]).reshape(4,1)
np.matmul(Jacobian(), delta)

array([[-3.061617e-16],
       [-5.000000e+00],
       [ 3.061617e-16]])